# Init for pyqcu.

In [1]:
import cupy as cp
import numpy as np
import functools
from pyqcu import define, io, qcu, eigen, cg, bistabcg, amg, linalg, gauge, demo
from time import perf_counter
from opt_einsum import contract
from pyqcu.set import params,set_ptrs,argv
params[define._LAT_X_] = 4
params[define._LAT_Y_] = 4
params[define._LAT_Z_] = 4
params[define._LAT_T_] = 8
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_] * params[define._LAT_Z_] * params[define._LAT_T_]
params[define._DATA_TYPE_] = define._LAT_C64_
sigma=0.1
seed=12138



    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

In [2]:
print('My rank is ', define.rank)
print("Parameters:", params)
print("Arguments:", argv)
wilson_cg_params = params.copy()
wilson_cg_params[define._SET_INDEX_] = 0
wilson_cg_params[define._SET_PLAN_] = define._SET_PLAN1_
qcu.applyInitQcu(set_ptrs, wilson_cg_params, argv)
wilson_dslash_eo_params = params.copy()
wilson_dslash_eo_params[define._SET_INDEX_] = 1
wilson_dslash_eo_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_params, argv)
wilson_dslash_eo_dag_params = params.copy()
wilson_dslash_eo_dag_params[define._SET_INDEX_] = 2
wilson_dslash_eo_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_dag_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_dag_params, argv)
wilson_dslash_oe_params = params.copy()
wilson_dslash_oe_params[define._SET_INDEX_] = 3
wilson_dslash_oe_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_params, argv)
wilson_dslash_oe_dag_params = params.copy()
wilson_dslash_oe_dag_params[define._SET_INDEX_] = 4
wilson_dslash_oe_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_dag_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_dag_params, argv)
print("Demo is running...")
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)
src = cp.zeros(int(params[define._LAT_XYZT_]*define._LAT_SC_ /
               define._LAT_P_), dtype=define.dtype(params[define._DATA_TYPE_]))
src = linalg.initialize_random_vector(src)
src = io.fermion2sctzyx(src, params)
print("Src data:", src.data)
print("Src shape:", src.shape)
dest = cp.zeros_like(src)
U = gauge.give_gauss_SU3(sigma=sigma, seed=seed,
                         dtype=define.dtype(params[define._DATA_TYPE_]), size=params[define._LAT_XYZT_]*define._LAT_D_)
U = io.gauge2dptzyxcc(U, params)
U = io.dptzyxcc2ccdptzyx(U)
print("U data:", U.data)
print("U shape:", U.shape)
# qcu.applyWilsonBistabCgQcu(dest, src,
#                            U, set_ptrs, wilson_cg_params)
# print("Dest data:", dest.data)
# print("Dest shape:", dest.shape)

My rank is  0
Parameters: [    4     4     4     8   512     1     1     1     1     0     0     1
     0 10000     3     0     0     4     4     4     8    24]
Arguments: [0.e+00 1.e-09]
gridDim.x               :2
blockDim.x              :128
host_params[_LAT_X_]    :2
host_params[_LAT_Y_]    :4
host_params[_LAT_Z_]    :4
host_params[_LAT_T_]    :8
host_params[_LAT_XYZT_] :256
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :8
lat_2dim[_XZ_]          :8
lat_2dim[_XT_]          :16
lat_2dim[_YZ_]          :16
lat_2dim[_YT_]          :32
lat_2dim[_ZT_]          :32
lat_3dim[_YZT_]         :128
lat_3dim[_XZT_]         :64
lat_3dim[_XYT_]    

In [ ]:

U, src, dest, set_ptrs, wilson_cg_params, wilson_dslash_eo_params, \
    wilson_dslash_oe_params, wilson_dslash_eo_dag_params,\
        wilson_dslash_oe_dag_params = demo.give(params)

In [ ]:
sigma = 1.0
dtype = cp.complex128
seed = 1238
U, a = gauge.give_gauss_su3(sigma=sigma, dtype=dtype, seed=seed)
print(" random coefficient a_i =", a.get())
print(" generated SU(3) matrix U =\n", U.get())
gauge.test_su3(U)
U.shape


# Read from hdf5 files.

In [ ]:
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_out_filename)
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
# eigenvalues_filename = gauge_filename.replace("gauge", "eigenvalues")
# print("Eigenvalues filename:", eigenvalues_filename)
# eigenvalues = io.hdf5_xxx2xxx(file_name=eigenvalues_filename)
# print("Eigenvalues data:", eigenvalues.data)
# print("Eigenvalues shape:", eigenvalues.shape)
# eigenvectors_filename = gauge_filename.replace("gauge", "eigenvectors")
# print("Eigenvectors filename:", eigenvectors_filename)
# eigenvectors = io.eigenvectors2sctzyx(
#     params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=eigenvectors_filename))
# print("Eigenvectors data:", eigenvectors.data)
# print("Eigenvectors shape:", eigenvectors.shape)
# testvectors_filename = gauge_filename.replace(
#     "gauge", "testvectors")
# print("Testvectors filename:", testvectors_filename)
# testvectors = io.eigenvectors2sctzyx(
#     params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=testvectors_filename))
# testvectors = io.xxxtzyx2mg_xxxtzyx(
#     testvectors, params)
# print("Testvectors data:", testvectors.data)
# print("Testvectors shape:", testvectors.shape)

# Run wilson bistabcg from pyqcu test.

In [ ]:
qcu.applyWilsonBistabCgQcu(fermion_out, fermion_in,
                           gauge, set_ptrs, wilson_cg_params)
# qcu.applyWilsonCgQcu(fermion_out, fermion_in,
#                            gauge, set_ptrs, wilson_cg_params)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))

# Give CG & BISTABCG Dslash.
> src_o-set_ptr->kappa()**2*dslash_oe(dslash_eo(src_o))

In [ ]:
def pdslash_no_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_params)
    return src-kappa**2*tmp1


def pdslash_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_dag_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_dag_params)
    return src-kappa**2*tmp1


def cg_dslash(src):
    return pdslash_dag(pdslash_no_dag(src))

def dslash_no_dag(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        dest, src, gauge, set_ptrs, wilson_dslash_eo_params)
    return dest

def dslash_dag(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        dest, src, gauge, set_ptrs, wilson_dslash_eo_dag_params)
    return dest

def dslash(src):
    return dslash_no_dag(src)

def bistabcg_dslash(src):
    return pdslash_no_dag(src)

print(cp.linalg.norm((fermion_out[define._EVEN_]-kappa *
               dslash(fermion_out[define._ODD_]))-fermion_in[define._ODD_]))

# Give matvec.

In [ ]:
# def matvec(src):
#     return gamma5_vec(pdslash_no_dag(src)).reshape(src.shape)
def matvec(src):
    return cg_dslash(src).reshape(src.shape)
# def matvec(src):
#     return bistabcg_dslash(src).reshape(src.shape)

# AMG - SETUP

In [ ]:
# _, _testvectors = eigen.solver(n=fermion_in[define._EVEN_].size, k=params[define._LAT_E_],
#                                matvec=matvec, dtype=fermion_in.dtype)
# # _testvectors = amg.setup(n=fermion_in[define._EVEN_].size, k=params[define._LAT_E_],
# #                          matvec=matvec, dtype=fermion_in.dtype)
# testvectors = io.xxxtzyx2mg_xxxtzyx(
#     io.eigenvectors2sctzyx(_testvectors, params), params)
# print("Shape of testvectors: ", testvectors.shape)
# for i in range(len(_testvectors)):
#     if i == 0:
#         pass
#     else:
#         print("Ax/x", bistabcg_dslash(_testvectors[i])/_testvectors[i])
#         projections = cp.dot(_testvectors[:i].conj(), _testvectors[i])
#         max_proj = cp.max(cp.abs(projections)).get()
#         print(f"Maximum projection onto existing basis: {max_proj:.2e}")
#         j = i+1
#         if j == len(_testvectors):
#             j = 0
#         print("Difference between v_i and v_j:", cp.linalg.norm(
#             _testvectors[i]-_testvectors[j])/cp.linalg.norm(_testvectors[i]))
# for T in range(params[define._MG_T_]):
#     for Z in range(params[define._MG_Z_]):
#         for Y in range(params[define._MG_Y_]):
#             for X in range(params[define._MG_X_]):
#                 testvectors[:, :, :, T, :, Z, :, Y, :, X, :] = linalg.orthogonalize_matrix(
#                     testvectors[:, :, :, T, :, Z, :, Y, :, X, :].reshape(params[define._LAT_E_], -1)).reshape(testvectors[:, :, :, T, :, Z, :, Y, :, X, :].shape)

# io.xxx2hdf5_xxx(
#     testvectors, params, gauge_filename.replace("gauge", "testvectors"))

In [ ]:
testvectors = io.xxxtzyx2mg_xxxtzyx(
    cp.ones_like(fermion_in[define._EVEN_]), params)
testvectors /= (XX*YY*ZZ*TT)**0.5

# Verify $(\gamma_5 D)^\dag = D^\dag {\gamma_5}^\dag = D^\dag \gamma_5  = \gamma_5 D$

In [ ]:
gamma5 = cp.array([[1, 0, 0, 0], [0, 1, 0, 0], [
                   0, 0, -1, 0], [0, 0, 0, -1]]).astype(gauge.dtype)
print(gamma5)
print(gamma5.T)


def gamma5_vec(src):
    return contract("ss,sctzyx->sctzyx", gamma5, io.fermion2sctzyx(src, params))


def vec_gamma5(src):
    return contract("sctzyx,ss->sctzyx", io.fermion2sctzyx(src, params), gamma5)


_src = fermion_out[define._EVEN_]
print(_src.shape)
print(cp.linalg.norm(gamma5_vec(dslash_no_dag(_src))-dslash_dag(gamma5_vec(_src))))
print(cp.linalg.norm(gamma5_vec(pdslash_no_dag(_src))-pdslash_dag(gamma5_vec(_src))))

# MultiGrid - give grids.

In [ ]:
# _src = io.xxxtzyx2mg_xxxtzyx(
#     icput_array=fermion_in[define._EVEN_], params=params)
_src = io.xxxtzyx2mg_xxxtzyx(
    icput_array=fermion_out[define._EVEN_], params=params)

<!-- # MultiGrid - R*vector.
![](./image0-dev40.png) -->

In [ ]:
r_src = _src


def r_vec(src):
    if src.ndim == 1:
        return contract("scTtZzYyXx,scTtZzYyXx->scTZYX", cp.conj(testvectors), io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(src, params), params)).flatten()
    return contract("scTtZzYyXx,scTtZzYyXx->scTZYX", cp.conj(testvectors), src)


r_dest = r_vec(r_src)
p_src = r_dest


def p_vec(src):
    if src.ndim == 1:
        return contract("scTtZzYyXx,scTZYX->scTtZzYyXx", testvectors, io.xxx2scTZYX(src, params)).flatten()
    return contract("scTtZzYyXx,scTZYX->scTtZzYyXx", testvectors, src)


p_dest = p_vec(p_src)
_mat = contract("scTtZzYyXx,scTtZzYyXx->scTZYX",
                cp.conj(testvectors), testvectors).flatten()
print(r_src.flatten()[:50])
print(p_dest.flatten()[:50])
print(_mat[:50])
print(
    p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))).flatten()[:50])
print(cp.linalg.norm(r_src))
print(cp.linalg.norm(p_dest))
print(cp.linalg.norm(r_src)/cp.linalg.norm(p_dest))
print(cp.linalg.norm(_mat))
print(cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src))
print(cp.linalg.norm(r_src-p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src))))))))
                     )/cp.linalg.norm(r_src))

<!-- # MultiGrid - verify above.
![](./image2-dev40.png) -->

# MultiGrid - R*matvec\*P.

In [ ]:
def r_matvec_p(src, matvec):
    return r_vec(matvec(p_vec(src)))


D_r_src = matvec(r_src)
D_p_r_dest = matvec(p_vec(r_vec(r_src)))
p_r_D_p_r_dest = p_vec(r_vec(D_p_r_dest))
print(D_r_src.flatten()[:50])
print(D_p_r_dest.flatten()[:50])
print(p_r_D_p_r_dest.flatten()[:50])
print(cp.linalg.norm(D_r_src-D_p_r_dest)/cp.linalg.norm(D_r_src))
print(cp.linalg.norm(D_r_src-p_r_D_p_r_dest)/cp.linalg.norm(D_r_src))

# Give matvec_c

In [ ]:
matvec_c = functools.partial(r_matvec_p, matvec=matvec)

# AMG-2-LEVEL

## give b

In [ ]:
b_e = fermion_in[define._EVEN_].flatten()
b_o = fermion_in[define._ODD_].flatten()
b__o = cp.zeros_like(b_o)
tmp = cp.zeros_like(b_o)
# b__o=b_o+kappa*D_oe(b_e)
qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
b__o = b_o+kappa*tmp
# b__o -> Dslash^dag b__o
b__o = pdslash_dag(b__o)
b = b__o.copy()

## solver

### give x_a

In [ ]:
x_a = bistabcg.slover(
    b=b, matvec=matvec, tol=1e-1)

### iterate

In [ ]:
mg = True
for i in range(params[define._MAX_ITER_]):
    ######
    if i == 0:
        x_f = x_a.copy()
        r_f = b-matvec(x_f)
    if (mg):
        ######
        r_c = r_vec(r_f)
        ######
        if i == 0:
            e_c = cp.zeros_like(r_c)
        r_e_c = r_c-matvec_c(e_c)
        tol_c = linalg.norm2(r_e_c)
        print(f"CCCFFF Iteration {i}, tol_c={tol_c} FFFCCC")
        ######
        e_c = cg.slover(
            b=r_c, matvec=matvec_c, max_iter=20, x0=e_c)
        ######
        r_e_c = r_c-matvec_c(e_c)
        tol_c = linalg.norm2(r_e_c)
        print(f"CCCCCC Iteration {i}, tol_c={tol_c} CCCCCC")
        ######
        e_f = p_vec(e_c)
        ######
        x_f += e_f
        ######
    r_f = b-matvec(x_f)
    tol_f = linalg.norm2(r_f)
    print(f"FFFCCC Iteration {i}, tol_f={tol_f} CCCFFF")
    ######
    x_f = cg.slover(
        b=b, matvec=matvec, max_iter=10, x0=x_f)
    ######
    r_f = b-matvec(x_f)
    tol_f = linalg.norm2(r_f)
    print(f"FFFFFF Iteration {i}, tol_f={tol_f} FFFFFF")
    ######
    if tol_f < argv[define._TOL_]:
        break
    ######

## refer

In [ ]:
# x = cg.slover(
#     b=b__o, matvec=matvec, tol=1e-6)

## check

In [ ]:
x_o = x.copy()
qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
x_e = b_e+kappa*tmp
# give qcu_fermion_out
qcu_fermion_out = cp.zeros_like(quda_fermion_out)
qcu_fermion_out[define._EVEN_] = x_e.reshape(
    quda_fermion_out[define._EVEN_].shape)
qcu_fermion_out[define._ODD_] = x_o.reshape(
    quda_fermion_out[define._ODD_].shape)
print(cp.linalg.norm(qcu_fermion_out-quda_fermion_out) /
      cp.linalg.norm(quda_fermion_out))

# MG-BISTABCG

In [ ]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # # Dslash(x_o)=b__o
# x_o = bistabcg.slover(
#     b=b__o, matvec=matvec, tol=1e-6)
# # io.xxx2hdf5_xxx(x_o, params, 'x_o.h5')
# # mg version
# mg_b__o = r_vec(io.xxxtzyx2mg_xxxtzyx(
#     io.fermion2sctzyx(b__o, params), params)).flatten()
# mg_x_o = bistabcg.slover(
#     b=mg_b__o, matvec=functools.partial(r_matvec_p, matvec=matvec), tol=1e-6)
# _x_o = io.array2xxx(p_vec(io.xxx2scTZYX(mg_x_o, params)))
# # io.xxx2hdf5_xxx(_x_o, params, '_x_o.h5')
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(cp.linalg.norm(qcu_fermion_out-quda_fermion_out) /
#       cp.linalg.norm(quda_fermion_out))
# # x_o = io.hdf5_xxx2xxx(params, 'x_o.h5')
# # _x_o = io.hdf5_xxx2xxx(params, '_x_o.h5')
# print(x_o.flatten()[:50])
# print(_x_o.flatten()[:50])
# print(cp.linalg.norm(_x_o-x_o) /
#       cp.linalg.norm(x_o))

# End for pyqcu. (pass, don't run this)

In [ ]:
# qcu.applyEndQcu(set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_dag_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_dag_params)